In [3]:
#to access kaggle datasets
!pip install kaggle

In [4]:
# Colab's file access feature
from google.colab import files

#retrieve uploaded file
uploaded = files.upload()

#print results
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 65 bytes


In [5]:
!kaggle datasets download -d sid321axn/fakernewsdata/download/fakernewsdata.zip/1

 80% 33.0M/41.0M [00:00<00:00, 29.5MB/s]
100% 41.0M/41.0M [00:00<00:00, 44.4MB/s]


In [6]:
!ls
!unzip fakernewsdata.zip
!ls

fakernewsdata.zip  sample_data
Archive:  fakernewsdata.zip
  inflating: fake_news.csv           
fake_news.csv  fakernewsdata.zip  sample_data


In [0]:
import pandas as pd
df = pd.read_csv('fake_news.csv')

In [8]:
df.head()

,Unnamed: 0,title,text,label
0,0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,TRUE
1,1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,TRUE
2,2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,TRUE
3,3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,TRUE
4,4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,TRUE


In [9]:
import numpy as np
df =df.reindex(np.random.permutation(df.index))
df.head()

,Unnamed: 0,title,text,label
40925,40925,WOW! JILL STEIN’S ‘FIRESIDE CHAT’ Exposes Her ...,,FAKE
29304,29304,Donald Trump Just Said The DUMBEST Thing Abou...,Donald Trump made a pathetic appearance at Reg...,FAKE
16622,16622,EU's Tusk says 'good deal' or 'no deal' on Bre...,BRUSSELS (Reuters) - European Council Presiden...,TRUE
28554,28554,Rosie O’Donnell Writes Powerful Poem That Per...,If anyone knows what it s like to be attacked ...,FAKE
40764,40764,BOOM! Trump Drains Obama Swamp…Mandates All Am...,President-elect Donald J. Trump s transition s...,FAKE


In [10]:
df.drop(['Unnamed: 0'],axis=1, inplace=True)
df["label"] = df.label.apply(lambda  x:0 if x=='TRUE' else 1)
df.head()


,title,text,label
40925,WOW! JILL STEIN’S ‘FIRESIDE CHAT’ Exposes Her ...,,1
29304,Donald Trump Just Said The DUMBEST Thing Abou...,Donald Trump made a pathetic appearance at Reg...,1
16622,EU's Tusk says 'good deal' or 'no deal' on Bre...,BRUSSELS (Reuters) - European Council Presiden...,0
28554,Rosie O’Donnell Writes Powerful Poem That Per...,If anyone knows what it s like to be attacked ...,1
40764,BOOM! Trump Drains Obama Swamp…Mandates All Am...,President-elect Donald J. Trump s transition s...,1


In [0]:
spec_chars = ["!",'"',"#","%","&","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–"]
for char in spec_chars:
    df['title'] = df['title'].str.replace(char, ' ')

In [12]:
df['title']=df['title'].str.lower()
df.head()

,title,text,label
40925,wow jill stein’s ‘fireside chat’ exposes her ...,,1
29304,donald trump just said the dumbest thing abou...,Donald Trump made a pathetic appearance at Reg...,1
16622,eu s tusk says good deal or no deal on bre...,BRUSSELS (Reuters) - European Council Presiden...,0
28554,rosie o’donnell writes powerful poem that per...,If anyone knows what it s like to be attacked ...,1
40764,boom trump drains obama swamp…mandates all am...,President-elect Donald J. Trump s transition s...,1


In [13]:
!pip3 install ktrain

     |████████████████████████████████| 25.2MB 135kB/s 
     |████████████████████████████████| 421.8MB 40kB/s 
     |████████████████████████████████| 983kB 15.5MB/s 
     |████████████████████████████████| 245kB 39.4MB/s 
     |████████████████████████████████| 645kB 30.8MB/s 
     |████████████████████████████████| 471kB 31.9MB/s 
     |████████████████████████████████| 450kB 38.0MB/s 
     |████████████████████████████████| 3.9MB 18.9MB/s 
     |████████████████████████████████| 1.0MB 30.0MB/s 
     |████████████████████████████████| 890kB 36.7MB/s 
     |████████████████████████████████| 3.8MB 35.4MB/s 
  Created wheel for ktrain: filename=ktrain-0.14.7-cp36-none-any.whl size=25240969 sha256=e0806742c0019120bfdc63cc5d10e72116751adc6d4c0f32ba93dfd4137ea3eb
  Stored in directory: /root/.cache/pip/wheels/d6/99/6b/4fc1aad36f8231465508e10579ccf401b80ef0e5eb2ab9a17d
  Created wheel for keras-bert: filename=keras_bert-0.81.0-cp36-none-any.whl size=37913 sha256=6d4245caad8e7a1b4196bbc7d7f

In [0]:
import ktrain
from ktrain import text
import numpy as np # linear algebra
from sklearn.model_selection import train_test_split
# fetch the dataset using scikit-learn
categories = ['TRUE', 'FAKE']

y = df['label'].values
X = df['title']

X_train, X_test, y_train, y_test = train_test_split(X, y,stratify=y,test_size=0.30, random_state=0)

In [18]:
(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=X_train, y_train=y_train,
                                                                       x_test=X_test, y_test=y_test,
                                                                       class_names=categories,
                                                                       preprocess_mode='bert',
                                                                       maxlen=350, 
                                                                       max_features=35000)

task: text classification
preprocessing train...
language: en


preprocessing test...
language: en


In [19]:
# you can disregard the deprecation warnings arising from using Keras 2.2.4 with TensorFlow 1.14.
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)
learner = ktrain.get_learner(model, train_data=(x_train, y_train), batch_size=6)

Is Multi-Label? False
maxlen is 350
done.


In [20]:
learner.fit_onecycle(2e-5, 1)



begin training using onecycle policy with max lr of 2e-05...
Train on 31428 samples
31428/31428 [==============================] - 2782s 89ms/sample - loss: 0.1111 - accuracy: 0.9570


We can use the learner.validate method to test our model against the Test set.

In [21]:
learner.validate(val_data=(x_test, y_test), class_names=categories)

              precision    recall  f1-score   support

        TRUE       0.98      0.99      0.98      6425
        FAKE       0.99      0.98      0.98      7045

    accuracy                           0.98     13470
   macro avg       0.98      0.98      0.98     13470
weighted avg       0.98      0.98      0.98     13470



array([[6356,   69],
       [ 143, 6902]])

## How to Use Our Trained BERT Model
We can call the learner.get_predictor method to obtain a Predictor object capable of making predictions on new raw data.

In [0]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [23]:
predictor.get_classes()

['TRUE', 'FAKE']